<a href="https://colab.research.google.com/github/AbiSyaibah/Data-Science/blob/main/World%20Cup%20Predict/WorldCup_Prediction_(Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [4]:
dict_table = pickle.load(open('dict_table','rb'))
historical_data=pd.read_csv("fifa_worldcup_clean_data.csv")
fixture_data=pd.read_csv('fifa_worldcup_clean_fixture.csv')

In [5]:
historical_data

,Team Tuan Rumah,Team Tamu,Tahun Piala Dunia,Home Goal,Away Goal,Total Goal
0,Uruguay,Argentina,1930,4,2,6
1,France,Mexico,1930,4,1,5
2,Argentina,France,1930,1,0,1
3,Chile,Mexico,1930,3,0,3
4,Chile,France,1930,1,0,1
...,...,...,...,...,...,...
895,Belgium,Panama,2018,3,0,3
896,Mexico,Sweden,2018,0,3,3
897,South Korea,Germany,2018,2,0,2
898,South Korea,Mexico,2018,1,2,3


#1. Mengukur kekuatan team

In [6]:
#Split data menjadi df_home dan df_away
df_home=historical_data[["Team Tuan Rumah","Home Goal","Away Goal"]]
df_away=historical_data[["Team Tamu","Home Goal","Away Goal"]]

In [7]:
#rename columns
df_home=df_home.rename(columns={"Team Tuan Rumah":"Team","Home Goal":"GoalsScored","Away Goal":'GoalsConceded'})
df_away=df_away.rename(columns={"Team Tamu":"Team","Home Goal":"GoalsConceded","Away Goal":'GoalsScored'})

In [8]:
#menggabungkan kedua data(concat)
df_team_strength=pd.concat([df_home,df_away],ignore_index=True)

In [9]:
#menggrupkan data(groupby) dan melihat mean-nya(mean)
df_team_strength=df_team_strength.groupby('Team').mean()

In [10]:
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


#2. Fungsi Predict_points

In [11]:
home='Argentina'
df_team_strength.at[home,'GoalsScored']

1.691358024691358

In [12]:
def predict_point(home,away):
  #jika home dan away ada dalam index df_team_strength
  if home in df_team_strength.index and away in df_team_strength.index:
      #goal scored * goal conceded
      lamb_home=df_team_strength.at[home,'GoalsScored']*df_team_strength.at[away,'GoalsConceded']
      lamb_away=df_team_strength.at[away,'GoalsScored']*df_team_strength.at[home,'GoalsConceded']
      prob_home, prob_away,prob_draw=0,0,0
      #kenapa dibuat dari 0-11, karena goal yang dapat terjadi kita batasi jadi maksimal 11 goal
      for x in range(0,11):  #jumlah goal tim away
          for y in range(0,11):   #jumlah goal tim home
          # x dan y adalah jumlah goal team (iterasi akan terlihat sebagai (1-0, 1-1,1-2,...))
          # p adalah probability terjadinya goal (yang dicari menggunakan persamaan poison)
              p=poisson.pmf(x, lamb_home) * poisson.pmf(y,lamb_away)
              if x==y:
                prob_draw += p
              elif x>y:
                prob_home += p
              else:
                prob_away += p
      
      # formula to calculate the strength of a team
      points_home=3*prob_home+prob_draw
      points_away=3*prob_away+prob_draw

      return (points_home, points_away)
  else:
      return(0,0)

In [13]:
#Test the function
predict_point('Argentina', 'Portugal')

(1.4520553471618411, 1.3367139548380762)

#3. Predicting World Cup

**3.1 Group Stage**

In [14]:
#splitting fixture into group, knockout, quarter,...
fixture_group_stage=fixture_data[:48].copy()
fixture_knockout_stage=fixture_data[48:56].copy()
fixture_quarter_final=fixture_data[56:60].copy()
fixture_semi_final=fixture_data[60:62].copy()
fixture_final=fixture_data[62:].copy()

In [15]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [16]:
#run all the matches in group stage and update grup table
for group in dict_table:
  teams_in_group=dict_table[group]['Team'].values
  df_fixture_group_6=fixture_group_stage[fixture_group_stage['home'].isin(teams_in_group)]
  for index,row in df_fixture_group_6.iterrows():
    home,away = row['home'], row['away']
    points_home, points_away= predict_point(home,away)
    dict_table[group].loc[dict_table[group]['Team']==home,'Pts'] += points_home
    dict_table[group].loc[dict_table[group]['Team']==away,'Pts'] += points_away

  dict_table[group]=dict_table[group].sort_values('Pts',ascending=False).reset_index()
  dict_table[group]=dict_table[group][['Team', 'Pts']]
  dict_table[group]=dict_table[group].round(0)
    

In [17]:
dict_table['Group H']

,Team,Pts
0,Portugal,6.0
1,Uruguay,5.0
2,Ghana,4.0
3,South Korea,2.0


**3.2 Knock Out**

In [18]:
#fixture_knockout_stage
fixture_knockout_stage

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [19]:
#update knockout fixture dengan data pemenang dan runner up tiap grup
for group in dict_table:
  group_winner=dict_table[group].loc[0,'Team']
  runners_up=dict_table[group].loc[1,'Team']
  fixture_knockout_stage.replace({f'Winners {group}':group_winner,f'Runners-up {group}':runners_up},inplace=True)

fixture_knockout_stage['winner']='?'

In [20]:
fixture_knockout_stage

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [21]:
# create winner function
def get_winner(fixture_knockout_stage):
  for iter,row in fixture_knockout_stage.iterrows():
    home, away= row['home'], row['away']
    points_home, points_away=predict_point(home, away)
    if points_home>points_away:
      winner=home
    else:
      winner=away
    fixture_knockout_stage.loc[iter,'winner']=winner
  return fixture_knockout_stage   

In [22]:
get_winner(fixture_knockout_stage)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


In [23]:
fixture_knockout_stage

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


**3.3 Quarter Final**

In [27]:
# update fixture_quarter_final
def update_table(fixture_knockout_stage,fixture_quarter_final):
  for iter,row in  fixture_knockout_stage.iterrows():
    winner=fixture_knockout_stage.loc[iter,'winner']
    match=fixture_knockout_stage.loc[iter,'score']
    fixture_quarter_final.replace({f'Winners {match}':winner}, inplace=True)
  fixture_quarter_final['winner']='?'
  return fixture_quarter_final

In [25]:
fixture_quarter_final

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [26]:
get_winner(fixture_quarter_final)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


**3.4 Semifinal**

In [28]:
update_table(fixture_quarter_final,fixture_semi_final)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [31]:
get_winner(fixture_semi_final)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


**3.5 Final**

In [32]:
update_table(fixture_semi_final,fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [33]:
# update third place match
fixture_final.replace({'Losers Match 61':'Netherlands','Losers Match 62':'Portugal'},inplace=True)

In [35]:
fixture_final

,home,score,away,year,winner
62,Netherlands,Match 63,Portugal,2022,?
63,Brazil,Match 64,France,2022,?


In [36]:
# mendapatkan pemenang pildun
get_winner(fixture_final)

,home,score,away,year,winner
62,Netherlands,Match 63,Portugal,2022,Netherlands
63,Brazil,Match 64,France,2022,Brazil
